In [3]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
from sklearn.model_selection import train_test_split
import pandas as pd
import os

/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [43]:
csv_file = 'instagram_data.csv'
df = pd.read_csv(csv_file)


print(df.head())

    likes  no_of_comments           t  follower_count_at_t  \
0  154552               0  1594174009             40934474   
1   97386               0  1593571666             40934474   
2  145632               0  1593136341             40934474   
3   76461               0  1592981047             40934474   
4  174620               0  1592703461             40934474   

                 image_path  
0  ../Data/insta_data/0.jpg  
1  ../Data/insta_data/2.jpg  
2  ../Data/insta_data/4.jpg  
3  ../Data/insta_data/6.jpg  
4  ../Data/insta_data/8.jpg  


In [13]:
from instagram_dataset import InstagramDataset

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = InstagramDataset(csv_file='instagram_data.csv', root_dir='insta_data', transform=transform)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)



for images, likes in train_loader:
    print(f'Batch of train_images has shape: {images.shape}')
    print(f'Batch of train_likes: {likes}')
    
    break  
for images, likes in test_loader:
    print(f'Batch of test_images has shape: {images.shape}')
    print(f'Batch of test_likes: {likes}')
    
    break  


/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

61983
55126
441330
228956
154357
363021
241835
443905
129443
264626
677191
5684
173958
31656
365000
39615
40850
8134
279475
167540
27623
17607
429869
386290
40303
129168
76846
59448
81941
41397
23717
11570
50191
120306
47763
326242
283340
221741
499003
100700
216502
192785
90911
159857
47588
69290
211138
188417
72419
793023
93909
228217
29238
67732
110075
168809
362227
14647
80001
34256
170854
79891
57111
38772
59339
36975
644145
574494
324061
487005
113752
274363
86397
63004
67454
254674
291171
175560
104763
288862
332554
50171
260948
18187
239902
235450
139844
293611
89191
88036
132765
317836
86867
208762
264034
218801
906701
8516
170454
216536
39002
177394
20149
163302
404134
290775
248630
125862
327479
209741
67410
48844
393208
66528
60600
7555
171103
331993
434391
37134
154254
342133
52671
Batch of train_images has shape: torch.Size([32, 3, 224, 224])
152296
Batch of train_likes: tensor([441330., 154357., 443905., 677191.,  31656., 279475.,  17607.,  40303.,
         59448.,  1157

/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

36490
203713
18715
276746
376486
102396
40161
395880
60359
295230
34551
235976
9079
384996
327047
23932
763782
339820
171717
355132
21670
83403
181467
195352
153409
41852
58555
173865
77685
43921
8667
4090
45766
9560
133738
193128
399930
9854
74543
350474
16152
151485
189545
30477
335173
14617
41633
18794
49115
212159
3929
229759
4642
12769
75943
182433
724854
20691
365595
356872
131955
121716
312876
175043
117271
278524
529914
64277
495984
64371
395520
45404
49324
149060
340127
114900
44432
78348
162864
5697
14254
247612
123644
240886
192693
402224
45578
84997
72375
421947
892989
316424
288247
135450
223296
181406
59587
1187749
564608
100446
293427
84585
10044
228183
51343
215299
130480
116363
24073
485190
32276
61445
147981
31530
24921
24962
789036
7889
113804
88240
Batch of test_images has shape: torch.Size([32, 3, 224, 224])
Batch of test_likes: tensor([ 18715., 102396., 395880.,  34551.,   9079.,  23932., 171717.,  21670.,
        195352.,  58555.,  43921.,   9560., 193128.,  7454

In [9]:
resnet18 = models.resnet18(pretrained=True)
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)
num_ftrs = resnet18.fc.in_features
# resnet18.fc = nn.Linear(num_ftrs, 1)
# early_stopping = EarlyStopping(patience=7, verbose=True, path='resnet18_checkpoint.pth')
resnet18.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),
    nn.ReLU(),
    nn.Dropout(0.5), 
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 1)  
)

/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
criterion = nn.MSELoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)

In [49]:
num_epochs = 10

resnet18.train()

for epoch in range(num_epochs):
    running_loss = 0.0

    for inputs, labels in train_loader:

        # inputs = inputs.to(device)
        optimizer.zero_grad()

        
        outputs = resnet18(inputs)
        loss = criterion(outputs.squeeze(), labels)

       
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

111259
365177
58922
370350
387135
162864
193792
6984
85031
26274
113752
182832
65162
80297
113943
41633
31274
556288
109515
193385
642708
18562
56571
177394
11448
791625
67458
47271
179800
58880
235450
44663
258409
207044
85543
198168
272019
424640
54687
43262
179154
87154
157894
285614
8516
465086
11351
150217
261694
194580
224885
83477
580665
199936
133819
31173
337043
307388
225757
66355
93864
237179
154695
52934
119076
54662
134650
493674
9261
144042
77685
49510
121625
136984
356872
92537
63979
54137
181030
451093
1197266
59448
208002
148658
99750
98787
243144
183025
65093
113480
165630
63606
32645
24195
345211
39811
372866
70848
14254
178109
251924
27706
22887
6603
504794
1127774
16232
262258
267441
25686
69010
248113
11482
94666
38370
289501
72671
235233
385219
404134
6277
107216
163185
77395
127576
38502
150491
59860
67454
122460
293611
65631
64193
84818
34551
378753
56788
65325
474096
147354
120982
44994
19247
53355
332446
27105
77331
18644
40247
32369
307128
339246
159857
2204

/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

196187
74651
100212
68332
28582
197367
374861
111273
272532
682908
24018
116100
464848
110075
59734
405532
67083
212775
131955
32988
52758
108280
174707
225405
214581
141821
261787
36207
1347694
240886
178181
39811
62818
125844
12424
32821
72806
137465
139267
385914
23717
100290
281580
196764
246375
38482
6277
223801
261694
69590
600125
286467
9010
71389
279892
146220
156250
74909
275276
150189
296529
157590
137600
458507
145124
194628
196515
74325
38779
177635
81162
227509
131008
119076
123458
38944
314744
246653
28383
736407
49713
40622
257423
136116
672426
388110
521931
1023273
146179
57438
395359
31656
365177
57589
391038
79645
78729
638550
49226
293140
58880
288205
199070
143395
22887
219188
95205
606222
73189
114370
23859
41503
14567
122335
183768
313033
111218
56437
61584
154596
389243
153121
53587
64666
796695
80656
64413
76873
13608
59860
36380
22427
31694
272786
55557
26045
120089
156899
13001
49510
118789
570423
92679
56428
36265
40973
76741
23294
48145
29770
331993
215717
1

/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

...

429869
145637
1007986
493674
5317
41732
26274
49510
40850
287985
272786
132922
18028
210921
44936
65676
211425
319487
674843
96701
149295
55215
170558
428106
144125
622198
174800
264572
41691
41852
189564
225857
58173
133421
54041
529689
40973
247205
141243
84843
69612
201127
20978
25663
38747
341895
85031
77395
265904
30553
71585
391038
131791
207494
43172
56695
722937
42583
24924
3828
365191
661618
148034
118436
171479
54123
64413
49713
464928
64576
273131
393208
197227
193400
288786
35919
98925
351291
146827
353624
29845
214929
44905
509250
24812
301075
882532
109820
122193
513249
21920
11798
189091
46137
41702
177796
246747
27780
370350
64627
179296
56660
120089
123181
300393
739962
7889
46349
309648
57408
201772
95184
192693
155747
471765
4751
23879
96043
195551
327396
64481
79891
335173
22837
173022
43425
39693
120982
104691
300060
54323
89996
25597
147259
27456
22288
25458
421947
313033
249383
255616
184393
47338
14329
135306
12552
134710
236017
543475
94741
261694
84655
197505


/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

235233
201127
200161
39992
116204
126968
18582
110527
558086
41956
197002
66568
582505
60872
96043
44019
38537
383220
191049
58495
45766
23301
482927
460281
61285
513921
436347
259398
707686
6912
238726
33671
152002
67083
233766
66493
105501
186174
457504
376996
248000
174707
471765
616456
46986
181406
170725
156294
58111
77163
199765
70898
33405
724854
294941
113480
329859
186422
3400
68929
181452
38190
220273
363708
292722
614741
47653
115543
184978
165585
315799
421947
36503
97237
176451
66481
43223
43586
40078
277416
278619
110741
65399
90488
50913
14894
89122
61231
411654
265152
114900
123458
186277
516727
123440
145637
8452
34584
256267
288220
53398
79891
370178
56403281747

44905
30430
79645
3088
4521
33421
145220
30477
192007
129122
22348
62997
344148
356872
20347
36424
195105
289799
158478
111259
70086
150217
13830
53065
19764
141821
276746
148867
25460
27842
40133
3538
648736
11134
40693
32750
203713
36687
416706
22675
193128
189229
37078
34515
3816
255616
374861
446472
21092

/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

321109
273698
31087
39041
154357
104691
166759
284118
179800
111090
170704
38190
86121
40428
449684
706600
67276
19328
45129
323311
32536
216820
219188
22616
116553
275604
1431
402224
61343
118356
114017
264572
101866
44167
124223
201951
475804
143395
109242
27975
26589
120089
37818
208495
320479
415395
281747
196187
403538
30430
92499
235450
16301
850381
8134
677191
139013
6603
100563
563962
20277
75766
477374
154265
1028026
144909
78845
41852
243551
103425
220495
1151090
49025
233503
63502
2781
74772
49142
97233
259398
331085
58968
19976
701095
67454
74909
58548
107357
229465
35801
365595
55126
108021
154773
393208
50623
131358
281950
155747
57312
113587
183768
232462
36380
225757
218497
65861
201772
31995
474096
139267
199361
41633
565120
86510
308542
63668
13523
101294
138962
83180
66595
153009
175560
135967
59134
22264
145637
65220
943244
36687
38910
134355
378944
3816
174083
29100
117271
276608
68055
178591
13225
17552
42474
34860
127231
306481
96701
128884
219977
45766
311900
18

In [11]:
resnet18.load_state_dict(torch.load('resnet18_model.pth'))

/var/folders/y2/syv04x3j05z_4gnkc5_ltz040000gn/T/ipykernel_54065/2896844164.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet18.load_state_dict(torch.load('resnet18

<All keys matched successfully>

In [15]:
# Evaluating the model on the test set
resnet18.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet18(inputs).squeeze()

        log_preds = torch.log(outputs)
        log_labels = torch.log(labels)

        lower_bound = log_labels * 0.8
        upper_bound = log_labels * 1.2

        within_range = (log_preds >= lower_bound) & (log_preds <= upper_bound)
        correct += within_range.sum().item()
        total += labels.size(0)

    accuracy = correct / total * 100
    print(f'Accuracy: {accuracy:.2f}%')

resnet18.eval()
test_relative_differences = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet18(inputs).squeeze()

        log_preds = torch.log(outputs)
        log_labels = torch.log(labels)

        relative_difference = torch.abs(log_preds - log_labels) / torch.abs(log_labels)
        test_relative_differences.append(relative_difference)

average_relative_difference = torch.cat(test_relative_differences).mean().item()
print(f'Average Relative Difference on Test Set: {average_relative_difference*100:.4f}%')

/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

36490
203713
18715
276746
376486
102396
40161
395880
60359
295230
34551
235976
9079
384996
327047
23932
763782
339820
171717
355132
21670
83403
195352
181467
41852
153409
58555
77685
173865
43921
4090
8667
45766
9560
133738
193128
399930
74543
9854
350474
16152
189545
151485
30477
335173
41633
14617
18794
3929
49115
212159
229759
4642
75943
12769
724854
182433
20691
121716
131955
356872
365595
312876
529914
117271
175043
278524
64277
395520
495984
64371
49324
45404
340127
149060
44432
114900
78348
5697
162864
14254
123644
247612
402224
192693
240886
45578
84997
72375
421947
892989
288247
316424
223296
135450
181406
1187749
59587
100446
564608
293427
84585
228183
10044
51343
116363
24073
130480
215299
485190
61445
147981
32276
24921
31530
24962
789036
7889
113804
88240
917598
47338
19829
56019
334749
48363
24884
61553
471765
60120
310705
118356
374397
154087
7443
177635
158478
75701
177826
43146
44181
40973
64462
68639
35990
6399
25458
207044
275276
11473
8322
3088
189658
244066
512162


/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <DEA8ABD3-1D38-30D9-B074-BE964B4E191C> /Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/xuwei/anaconda3/envs/pytorch/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlop

36490
203713
18715
276746
376486
102396
40161
60359
395880
295230
235976
34551
384996
327047
9079
23932
763782
339820
355132
171717
83403
21670
181467
195352
41852
153409
58555
77685
173865
4090
8667
43921
45766
133738
9560
399930
193128
9854
74543
350474
16152
30477
189545
151485
335173
41633
14617
18794
212159
3929
49115
4642
229759
12769
75943
724854
182433
20691
121716
131955
356872
365595
312876
529914175043

117271
278524
64277
495984
395520
64371
49324
149060
340127
45404
44432
114900
162864
5697
78348
123644
14254
247612
240886
402224
45578
192693
84997
421947
72375
892989
316424
288247
223296
181406
135450
1187749
564608
59587
100446
293427
10044
84585
228183
51343
215299
116363
24073
130480
485190
32276
147981
61445
31530
24921
24962
789036
7889
113804
88240
917598
47338
19829
56019
48363
334749
24884
471765
118356
61553
310705
60120
7443
374397
154087
75701
177635
177826
158478
43146
40973
68639
44181
64462
6399
35990
207044
25458
275276
11473
189658
8322
3088
244066
512162


In [51]:
torch.save(resnet18.state_dict(), 'resnet18_model.pth')